In [1]:
import json
from time import time
import pandas as pd
from pathlib import Path
from dpmm.pipelines import PrivBayesPipeline

/tmp/ipykernel_6555/3077299844.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Load data

In [3]:
wine_dir = Path().parent / "wine"

df = pd.read_pickle(wine_dir / "wine.pkl.gz")
with (wine_dir / "wine_bounds.json").open("r") as f:
    domain = json.load(f)

df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4192,white,6.3,0.21,0.29,11.7,0.048,49.0,147.0,0.99482,3.22,0.38,10.8,0
2157,white,5.9,0.21,0.24,12.1,0.044,53.0,165.0,0.99690,3.25,0.39,9.5,0
631,red,10.4,0.28,0.54,2.7,0.105,5.0,19.0,0.99880,3.25,0.63,9.5,0
3410,white,7.6,0.38,0.20,3.4,0.046,9.0,116.0,0.99440,3.15,0.41,9.4,0
3117,white,8.4,0.23,0.49,7.8,0.035,22.0,95.0,0.99350,3.04,0.34,12.0,1


# Build pipeline

In [4]:
pipeline = PrivBayesPipeline(
    epsilon=1,  # Privacy budget for generate model
    proc_epsilon=0.1,  # Privacy budget for data processing 
    binner_type="priv-tree",  # Discretisation algorithm
    gen_kwargs={
        "n_iters": 5000,
        "degree": 2,
    },
    delta=1e-5,  # Delta Setting in (eps, delta) differential privacu
    compress=True,  # Domain Compression paramter
    max_model_size=80,  # Maximum Model Size in Megabytes
    n_bins="auto"  # Number of bins
)

# Fit step

In [5]:
print("Fitting PrivBayes pipeline...", end='\r')
start_time = time()
# The number of rows to generate
pipeline.fit(df, domain)
fit_time = time()

print(f"Fitting PrivBayes pipeline - Took {fit_time - start_time:.2f} seconds")

Fitting PrivBayes pipeline - Took 9.58 seconds


# Generate step

In [6]:
print("Generating synthetic data...", end="\r")
# Generate synthetic data
synth_df = pipeline.generate(df.shape[0])
gen_time = time()

print(f"Generating synthetic data - Took {gen_time - fit_time:.2f} seconds")

synth_df.head()

Generating synthetic data - Took 21.07 seconds


,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,5.972005,0.277117,0.264276,1.415403,0.049275,3.481346,309.942270,0.995157,3.317983,0.384114,10.292124,1
1,white,6.276097,0.278412,0.388142,2.760496,0.081847,7.411602,203.211448,0.996068,3.201561,0.607007,10.506393,1
2,red,6.328726,0.276828,0.148834,44.105013,0.022153,34.661868,214.240876,0.989358,3.203622,0.482347,12.951182,0
3,white,5.758842,0.382937,0.400143,34.762719,0.195950,19.497104,49.010611,0.989618,3.098091,0.798246,11.206725,0
4,white,7.175714,0.204521,0.318206,5.196298,0.031780,57.690296,71.290304,0.991893,3.080850,0.465819,11.382762,0
